# Tackle Noise with Error Correction

In [ ]:
# python -m venv qiskit_env
 
#  qiskit_env\Scripts\activate  

# pip install qiskit 
# pip install jupyter
# pip install qiskit-aer  
# pip install matplotlib 
# pip install qiskit-ibm-runtime


# Import necessary libraries from Qiskit

# QuantumCircuit: Used to build quantum circuits by applying quantum gates.
# assemble: Combines quantum circuits into a format that the simulator or quantum computer can process.
# Aer: Qiskit's simulator provider, which includes 'qasm_simulator' for running quantum circuits.
# transpile: Optimizes the quantum circuit for a specific backend.
# plot_histogram: Visualizes the measurement results in a histogram.
# CompleteMeasFitter, complete_meas_cal: Tools from Qiskit Ignis used for error mitigation.
from qiskit import QuantumCircuit, assemble, Aer, transpile
from qiskit.visualization import plot_histogram
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter, complete_meas_cal

# Step 1: Define the Quantum Circuit
# ----------------------------------
# QuantumCircuit(3, 3) initializes a circuit with 3 qubits and 3 classical bits.
qc = QuantumCircuit(3, 3)

# Apply gates to create an entangled state
qc.h(0)        # Apply Hadamard gate to qubit 0 to create superposition
qc.cx(0, 1)    # Apply CNOT gate with qubit 0 as control and qubit 1 as target
qc.cx(0, 2)    # Apply CNOT gate with qubit 0 as control and qubit 2 as target

# Measure qubits into classical bits to record results
qc.measure([0, 1, 2], [0, 1, 2])

# Step 2: Transpile the Circuit for a Specific Backend
# ----------------------------------------------------
# Transpilation optimizes the circuit based on the backend (here, 'qasm_simulator')
backend = Aer.get_backend('qasm_simulator')    # Select simulator backend for running the circuit
transpiled_qc = transpile(qc, backend)         # Transpile for optimization on the selected backend

# Step 3: Simulate the Circuit Without Noise Mitigation
# -----------------------------------------------------
# Assemble the transpiled circuit to prepare for execution, and specify shots (repetitions) for statistical accuracy
qobj = assemble(transpiled_qc, shots=1000)     # Assemble the transpiled circuit for execution, with 1000 shots
job = backend.run(qobj)                        # Run the job on the simulator
result = job.result()                          # Get the result from the executed job
counts = result.get_counts()                   # Get the count of each outcome (bitstring) from the result

# Step 4: Set Up and Run Measurement Error Mitigation
# ---------------------------------------------------
# Measurement error mitigation reduces the effect of measurement errors on the result

# 1. Generate calibration circuits for measurement error mitigation
# - complete_meas_cal returns calibration circuits and state labels.
# - qubit_list specifies which qubits to calibrate for measurement error.
cal_circuits, state_labels = complete_meas_cal(qubit_list=[0, 1, 2])

# 2. Run calibration circuits to get calibration results
# - These results help to calibrate and later filter out measurement errors.
cal_job = backend.run(assemble(cal_circuits, backend=backend))
cal_results = cal_job.result()

# 3. Use CompleteMeasFitter to create an error mitigation filter from calibration results
# - CompleteMeasFitter is initialized with calibration results and state labels.
# - The filter can later be used to "correct" measurement errors in the results.
meas_fitter = CompleteMeasFitter(cal_results, state_labels)

# Step 5: Apply Error Mitigation Filter
# -------------------------------------
# Use the measurement fitter to mitigate errors in the original counts
mitigated_counts = meas_fitter.filter.apply(counts)

# Display Results
# ---------------
print(f"Original Counts : {counts}")          # Print the raw, uncorrected counts
print(f"Mitigated Counts : {mitigated_counts}") # Print the counts after error mitigation

# Visualize Results
# -----------------
# plot_histogram displays measurement results for visual comparison.
# - Pass both original and mitigated counts with a legend to distinguish them.
plot_histogram([counts, mitigated_counts], legend=['Original', 'Mitigated'])


ImportError: Qiskit is installed in an invalid environment that has both Qiskit >=1.0 and an earlier version. You should create a new virtual environment, and ensure that you do not mix dependencies between Qiskit <1.0 and >=1.0. Any packages that depend on 'qiskit-terra' are not compatible with Qiskit 1.0 and will need to be updated. Qiskit unfortunately cannot enforce this requirement during environment resolution. See https://qisk.it/packaging-1-0 for more detail.